In [1]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import division

In [2]:
k = 2
L = 10
g = 9.81
m = 2

In [3]:
def spring_field(step = 0.01, filename = None):
    import matplotlib.animation as animation
    global pos,vel
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-5,70)
    plt.xlim(-5,70)
    
    points, = ax.plot([],[], linestyle='None', marker='o', markersize = 10.)
    
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    pos = np.mgrid[0:5,0:5]*10.
    vel = np.zeros(shape=(2,3,3))
    pos[0,2,2] = 40.

    
    def init():
        points.set_data([],[])
    
        return points, time_text
    
    def animate(t):
        global pos,vel
        mass = pos[:,1:4,1:4]
        north = pos[:,0:3,1:4]
        south = pos[:,2:5,1:4]
        west = pos[:,1:4,0:3]
        east = pos[:,1:4,2:5]

        spring_len = lambda direction: np.sqrt(np.sum((direction-mass)**2,axis = 0))
        spring_n = spring_len(north)
        spring_s = spring_len(south)
        spring_e = spring_len(east)
        spring_w = spring_len(west)
        update_north = (spring_n-L)*(mass-north)/spring_n
        update_south = (spring_s-L)*(mass-south)/spring_s
        update_east = (spring_e-L)*(mass-east)/spring_e
        update_west = (spring_w-L)*(mass-west)/spring_w
        update_vel = -step*k*(update_north + update_south + update_east + update_west)/m
        update_pos = step*vel
        
        pos[:,1:4,1:4] += update_pos
        vel += update_vel
        
        points.set_data(pos[0].ravel(),pos[1].ravel())
        time_text.set_text(time_template % (t*step))
        return points, time_text
    
    ani = animation.FuncAnimation(fig, animate, np.arange(400),
                              interval=step*10, blit=True, init_func=init)

    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

In [4]:
spring_field()

In [10]:
def spring_field_state(step = 0.01, filename = None):
    import matplotlib.animation as animation
    global state
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-5,70)
    plt.xlim(-5,70)
    
    points, = ax.plot([],[], linestyle='None', marker='o', markersize = 10.)
    
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    pos = np.mgrid[0:5,0:5]*10.
    vel = np.zeros_like(pos)
    pos[0,2,2] = 25.
    state = np.array([pos,vel])
    
    def init():
        points.set_data([],[])
    
        return points, time_text
    
    def animate(t):
        global state
        mass = state[0,:,1:4,1:4]
        north = state[0,:,0:3,1:4]
        south = state[0,:,2:5,1:4]
        west = state[0,:,1:4,0:3]
        east = state[0,:,1:4,2:5]

        spring_len = lambda direction: np.sqrt(np.sum((direction-mass)**2,axis = 0))
        spring_n = spring_len(north)
        spring_s = spring_len(south)
        spring_e = spring_len(east)
        spring_w = spring_len(west)
        update_north = (spring_n-L)*(mass-north)/spring_n
        update_south = (spring_s-L)*(mass-south)/spring_s
        update_east = (spring_e-L)*(mass-east)/spring_e
        update_west = (spring_w-L)*(mass-west)/spring_w      
        
        state[0] += step*state[1]
        state[1,:,1:4,1:4] += -step*k*(update_north + update_south \
                             + update_east + update_west)/m
        
        points.set_data(state[0,0].ravel(),state[0,1].ravel())
        time_text.set_text(time_template % (t*step))
        return points, time_text
    
    ani = animation.FuncAnimation(fig, animate, np.arange(400),
                              interval=step*10, blit=True, init_func=init)

    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

In [11]:
spring_field_state()

In [7]:
spring_len = lambda direction,ballz: np.sqrt(np.sum((direction-ballz)**2,axis = 0))
def update_fun(tn,x):
    mass = x[0,:,1:4,1:4]
    north = x[0,:,0:3,1:4]
    south = x[0,:,2:5,1:4]
    west = x[0,:,1:4,0:3]
    east = x[0,:,1:4,2:5]
    
    spring_n = spring_len(north,mass)
    spring_s = spring_len(south,mass)
    spring_e = spring_len(east,mass)
    spring_w = spring_len(west,mass)
    
    update_north = (spring_n-L)*(mass-north)/spring_n
    update_south = (spring_s-L)*(mass-south)/spring_s
    update_east = (spring_e-L)*(mass-east)/spring_e
    update_west = (spring_w-L)*(mass-west)/spring_w
    
    result = np.zeros_like(x)
    result[1,:,1:4,1:4] = -k*(update_north + update_south + \
                              update_east + update_west)/m

    result[0] = x[1]
    return result

In [8]:
def spring_field_state_function(step = 0.01, filename = None):
    import matplotlib.animation as animation
    global state
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-5,70)
    plt.xlim(-5,70)
    
    points, = ax.plot([],[], linestyle='None', marker='o', markersize = 10.)
    
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    pos = np.mgrid[0:5,0:5]*10.
    vel = np.zeros_like(pos)
    pos[0,2,2] = 45.
    pos[1,2,2] = 45.
    state = np.array([pos,vel])
    
    def init():
        points.set_data([],[])
    
        return points, time_text
    
    def animate(t):
        global state
        
        state += step*update_fun(t*step,state)
        
        points.set_data(state[0,0].ravel(),state[0,1].ravel())
        time_text.set_text(time_template % (t*step))
        return points, time_text
    
    ani = animation.FuncAnimation(fig, animate, np.arange(400),
                              interval=step*10, blit=True, init_func=init)

    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

In [9]:
spring_field_state_function()